# Extract and clean food prices over times using information compiled in the Item Price Diff 

### Libraries

In [1]:
import numpy as np 
import pandas as pd
import datetime
import math
import re

### Code

In [2]:
## Read food to calories docuements
## Information important for the next steps:
food_information =  pd.read_excel('../../1. Data available/ISHTM_Hadhreen/Food to Calories/food_names_and_calories.xlsx')
food_information['possible_name'] = [x.lower().strip(' ').replace(" ", '_') if type(x) == str else np.nan for x in food_information['possible_name']]
food_information['food_name'] = np.array([x.lower().strip(' ').replace(" ", '_') for x in food_information['food_name']])
potential_name = np.append(np.array(food_information['possible_name'].dropna()), np.array(food_information['food_name']))

In [3]:
excel_file = pd.ExcelFile('../../1. Data available/ISHTM_Hadhreen/Kitchen Admin Documents/Item Prices Diff.xlsx')
nb_sheets = len(excel_file.sheet_names)

In [4]:
final_results = pd.DataFrame(columns=["kitchen_id", "type_of_food", "date", "price"])
for nb_sheet in range(nb_sheets):
    df_price = pd.read_excel('../../1. Data available/ISHTM_Hadhreen/Kitchen Admin Documents/Item Prices Diff.xlsx', sheet_name=nb_sheet)
    ## First extract kichen_ID
    matches = df_price.apply(lambda x: x.astype(str).str.contains(r'\bKitchen ID\b', case=False, na=False))
    locations_kitchen_id = [(row_idx, col_idx) for row_idx, col in matches.iterrows() for col_idx, match in enumerate(col) if match]
    if(len(locations_kitchen_id) == 0):
        continue
    kitchen_id = df_price.iloc[locations_kitchen_id[0][0], locations_kitchen_id[0][1] +1]
    
    ## Than clean the rest
    matches = df_price.apply(lambda x: x.astype(str).str.contains(r'\bDescription\b', case=False, na=False))
    # Extract row and column indices
    locations_description = [(row_idx, col_idx) for row_idx, col in matches.iterrows() for col_idx, match in enumerate(col) if match]
    
    # Check for the word "TOTAL"
    matches = df_price.apply(lambda x: x.astype(str).str.contains(r'\Dec\b', case=False, na=False))
    # Extract row and column indices
    locations_dec = [(row_idx, col_idx) for row_idx, col in matches.iterrows() for col_idx, match in enumerate(col) if match]
    
    # Extract only information
    price_extract = df_price.iloc[locations_description[0][0]:,locations_description[0][1]:locations_dec[0][1]+2]
    #price_extract = price_extract.dropna(axis=1, how='all')
    
    #Change column name
    price_extract.columns = price_extract.iloc[0]
    price_extract = price_extract.drop(price_extract.index[0])
    
    # Rename columns
    price_extract.columns = ['description', 'unit', '01/07/2024', '15/07/2024', '01/08/2024', '15/08/2024',
                             '01/09/2024', '15/09/2024','01/10/2024', '15/10/2024', '01/11/2024', '15/11/2024',
                             '01/12/2024', '15/12/2024']
    
    ## Remove na column description
    price_extract = price_extract.dropna(subset=['description'])
    
    # now remove information not corresponding to food 
    # Filter to keep only English words
    price_extract['description'] = [[word.lower().strip(' ').replace(" ", '_') for word in pair if re.match(r'^[a-zA-Z\s]+$', word)] for pair in [x.split('/') for x in np.array(price_extract['description'])]]
    price_extract['description']  = [x[0] for x in price_extract['description']]
    
    # Only select lines when the description is corresponding to a food name (ex not transportation)
    price_extract = price_extract[price_extract['description'].isin(potential_name)]
    price_extract['description'] = [x if x in np.array(food_information['food_name']) else np.array(food_information[food_information['possible_name'] == x]['food_name'])[0] for x in price_extract['description']] 
    
    ##Transform the data into a long data
    price_extract["kitchen_id"] = kitchen_id
    
    ##Drop unit columns
    price_extract = price_extract.drop('unit', axis=1)
    
    # Reshape DataFrame from wide to long format
    df_long = pd.melt(
        price_extract,
        id_vars=["kitchen_id", "description"],
        var_name="date",
        value_name="price"
    )
    
    # Drop rows where price is NaN
    df_long = df_long.dropna(subset=["price"]).reset_index(drop=True)
    
    # Reorganize columns
    df_long = df_long[["kitchen_id", "description", "date", "price"]]
    
    # Rename columns for clarity
    df_long = df_long.rename(columns={"description": "type_of_food"})

    final_results = pd.concat([final_results, df_long], ignore_index=True)

In [5]:
## First remove kitchen ids with nan
final_results = final_results[final_results['kitchen_id'].notna()]
## Extract only the first element when there is a comma - split 
final_results['kitchen_id'] = [str(name).split(',')[0] for name in final_results['kitchen_id']]
## Upload kitchen_id_localisation file
kitchen_data = pd.read_excel('../output/kitchen_ids_cluster.xlsx')
## Merge final_results with this file
final_results = pd.merge(final_results, kitchen_data[['kitchen_code', 'kitchen_locality']], left_on='kitchen_id', right_on='kitchen_code', how='left') 
final_results = final_results[['type_of_food', 'date', 'price', 'kitchen_code','kitchen_locality']]

##Kitchen not in service
kitchen_to_be_removed = ['KH/JA/184', 'KH/JA/185', 'KH/JA/186', 'KH/JA/187', 'KH/JA/188', 'KH/JA/189', 'KH/JA/190', 'KH/JA/191',
                         'KH/JA/192', 'KH/JA/193', 'KH/JA/194', 'KH/JA/195', 'KH/JA/196', 'KH/JA/197', 'KH/JA/198', 'KH/JA/199',
                         'KH/JA/200', 'KH/JA/201', 'KH/JA/202', 'KH/JA/203', 'KH/JA/204', 'KH/JA/205', 'KH/JA/206', 'KH/JA/207',
                         'KH/JA/208', 'KH/JA/209', 'KH/JA/210', 'KH/JA/211', 'KH/JA/212', 'KH/JA/213', 'KH/JA/214', 'KH/JA/215',
                         'KH/JA/216', 'KH/JA/217', 'KH/JA/218', 'KH/JA/219', 'KH/JA/220', 'KH/JA/221', 'KH/JA/222', 'KH/JA/223',
                         'KH/JA/224', 'KH/JA/225', 'KH/JA/226', 'KH/JA/227', 'KH/JA/228', 'KH/JA/229', 'KH/JA/230', 'KH/JA/231',
                         'KH/JA/232', 'KH/JA/233', 'KH/JA/234', 'KH/JA/235', 'KH/JA/236', 'KH/JA/237', 'KH/JA/135', 'KH/JA/136',
                         'KH/JA/137', 'KH/JA/138', 'KH/JA/139', 'KH/JA/140', 'KH/JA/141', 'KH/JA/142', 'KH/JA/143', 'KH/JA/148']

# Remocve then from this list
final_results = final_results[~final_results['kitchen_code'].isin(kitchen_to_be_removed)]

## trasnform price into float
final_results = final_results.loc[final_results['price'] != '.']
final_results['price'] = final_results['price'].astype(float)

In [6]:
# Create a complete grid of type_of_food, date, and kitchen_locality
unique_foods = final_results["type_of_food"].unique()
unique_dates = ['01/07/2024', '15/07/2024', '01/08/2024', '15/08/2024',
                '01/09/2024', '15/09/2024','01/10/2024', '15/10/2024', 
                '01/11/2024', '15/11/2024', '01/12/2024', '15/12/2024']
unique_localities = kitchen_data["kitchen_locality"].unique()

complete_grid = pd.MultiIndex.from_product(
    [unique_foods, unique_dates, unique_localities],
    names=["type_of_food", "date", "kitchen_locality"]
).to_frame(index=False)
complete_grid = complete_grid.dropna(subset=["kitchen_locality"]).reset_index(drop=True)

final_results_filled = final_results.copy()
# Fill missing prices with group mean
final_results_filled["price"] = final_results_filled.groupby(
    ["kitchen_locality", "type_of_food", "date"]
)["price"].transform(
    lambda x: x.fillna(x.mean())
)
final_results_filled = final_results_filled.drop_duplicates()

# Merge the complete grid with the original data
# Drop duplicates from final_results_filled
final_results_filled = final_results_filled.drop_duplicates(
    subset=["type_of_food", "date", "kitchen_locality"]
)

# Ensure no mismatches in kitchen_locality values
final_results_filled["kitchen_locality"] = final_results_filled["kitchen_locality"].str.strip()
complete_grid["kitchen_locality"] = complete_grid["kitchen_locality"].str.strip()

# Merge the complete grid with the final results
df_full = complete_grid.merge(
    final_results_filled[["type_of_food", "date", "price", "kitchen_locality"]],
    on=["type_of_food", "date", "kitchen_locality"],
    how="left"
)

##Save the data into excel
df_full.to_excel('../output/clean_item_price_time.xlsx', index = False)   